# Imports

In [ ]:
import os
from collections import Counter, defaultdict
import importlib
import json
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['figure.figsize'] = (15, 5)

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
%run ../../utils/__init__.py
config_logging(logging.INFO)

In [ ]:
%run ../../datasets/common/constants.py

In [ ]:
%run -n ./nlp_in_chexpert_groups.py

# Utils

In [ ]:
# Load experiments
def load_experiments(dataset_name):
    exp_by_abn = {}
    errors = defaultdict(list)
    for abnormality in CHEXPERT_DISEASES[1:]:
        fname = f'{dataset_name}-{abnormality.replace(" ", "-").lower()}'
        if not exist_experiment_pickle(fname):
            errors['not-found'].append(fname)
            continue
        exp = load_experiment_pickle(fname)
        exp_by_abn[abnormality] = exp
        
    if len(errors['not-found']):
        print('Not found: ', errors['not-found'])
        
    return exp_by_abn

# Debug running experiments

In [ ]:
%run -n ./nlp_in_chexpert_groups.py

In [ ]:
dataset_info = init_dataset_info('iu')
dataset_info.name

In [ ]:
exp = init_experiment('Cardiomegaly', dataset_info)
exp_LO

In [ ]:
%%time

kwargs = {
    # 'metric': 'bleu',
    # 'metric': 'rouge',
    'metric': 'cider-IDF',
    'k_times': 500,
    # 'k_gts': 1,
    'max_n': 500,
}
exp.append(calc_score_matrices(exp.grouped_2, dataset_info, groups=(0, 1), **kwargs))
# exp.append(calc_score_matrices(exp.grouped, dataset_info, **kwargs))

In [ ]:
exp[-1].cube

# Plot experiments

## Plot one example

In [ ]:
exp = load_experiment_pickle('iu-cardiomegaly')
exp

In [ ]:
exp.results[-1].cube

In [ ]:
RESULT_I = 1
METRIC_I = 0

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plot_heatmap(exp, result_i=RESULT_I, metric_i=METRIC_I)

if len(exp.results) > 1:
    plt.subplot(1, 2, 2)
    plot_heatmap(exp, result_i=-2, metric_i=METRIC_I)

In [ ]:
plt.figure(figsize=(8, 6))

plt.subplot(2, 1, 1)
plot_hists(
    exp, [
        (0, 0), (0, 1),
        # (0, 0), (1, 0),
        # (1, 1), (0, 1),
        # (1, 0), (0, 0),
    ],
    result_i=RESULT_I, metric_i=METRIC_I,
    xlabel=False, bins=50, range=(0, 1),
    add_n_to_label=True,
)

plt.subplot(2, 1, 2)
plot_hists(
    exp, [
        (1, 1), (1, 0),
        # (1, 1), (0, 1),
        # (0, 0), (1, 0),
        # (-2, -1), (-1, -1),
    ],
    result_i=RESULT_I, metric_i=METRIC_I, add_n_to_label=True,
    title=False, bins=50)

## Plot all 4x4 matrices

In [ ]:
exp_by_abn_iu = load_experiments('iu')
len(exp_by_abn_iu)

In [ ]:
exp_by_abn = exp_by_abn_iu
abnormalities = CHEXPERT_DISEASES[1:]
metrics = ['bleu', 'rouge', 'cider-IDF']
groups = [0, 1]
# groups = [-2, 0, -1, 1]
sampler = 'random-gen_k500_n500'
# groups = list(sorted(groups))

In [ ]:
keydict = { metric: i for i, metric in enumerate(metrics) }
keydict

In [ ]:
cubes_df = pd.DataFrame(index=abnormalities, columns=metrics)

for abn in abnormalities:
    exp = exp_by_abn[abn]

    results = [
        r
        for r in exp.results
        if (sampler is None or r.sampler == sampler) and \
            (groups is None or sorted(r.groups) == sorted(groups))
    ]
    # Not necessary to sort
    # results = sorted(results, key=lambda x: keydict[x.metric])

    if len(results) != len(metrics):
        err = f'Diff amount of results than metrics: {abn}, {len(results)}, {len(metrics)}'
        print(err)
        continue
    
    for result in results:
        cubes_df.loc[abn, result.metric] = result.cube

cubes_df.isnull().any(axis=0).any(axis=0)

In [ ]:
cubes_df.head(2)

In [ ]:
TICKS = [KEY_TO_LABEL[k] for k in groups]

In [ ]:
metrics_plotable = [(0, 'bleu'), (3, 'bleu'), (0, 'rouge'), (0, 'cider-IDF')]

In [ ]:
def get_max_DEPRECATED(df, metrics):
    all_cubes = []
    for m in metrics:
        for abn in abns:
            for cube in cubes_df.loc[abn, m].values:
                all_cubes.append(cube[0])
                if m == 'bleu':
                    all_cubes.append(cube[3])
    all_cubes = np.array(all_cubes)
    return all_cubes.min(), all_cubes.max()

In [ ]:
def get_limits_per_abnormality(df, abns, metrics):
    cubes_per_abn = defaultdict(list)
    for abn in abns:
        for m in metrics:
            cube = df.loc[abn, m]
            cubes_per_abn[abn].append(cube[0])
            if m == 'bleu':
                cubes_per_abn[abn].append(cube[3])
    cubes_per_abn = {abn: np.array(l) for abn, l in cubes_per_abn.items()}
    limits_per_abn = {abn: (cube.min(), cube.max()) for abn, cube in cubes_per_abn.items()}
    return limits_per_abn

In [ ]:
# abns = CHEXPERT_DISEASES[1:]
abns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

In [ ]:
limits = get_limits_per_abnormality(cubes_df, abns, ['bleu', 'rouge'])
limits_cider = get_limits_per_abnormality(cubes_df, abns, ['cider-IDF'])
limits #, limits_cider

In [ ]:
# MIN_VALUE, MAX_VALUE = get_max(cubes_df, ['bleu', 'rouge'])
# MIN_CIDER, MAX_CIDER = get_max(cubes_df, ['cider-IDF'])
# MIN_VALUE, MAX_VALUE, MIN_CIDER, MAX_CIDER

In [ ]:
n_rows = len(abns)
n_cols = len(metrics_plotable)
plt.figure(figsize=(n_cols*5, n_rows*5))

for abn_i, abn in enumerate(abns):
    for metric_col_j, (metric_i, metric) in enumerate(metrics_plotable):
        cube = cubes_df.loc[abn, metric]
        
        include_cbar = (metric_col_j >= n_cols - 2) # rouge and CIDEr
        # cbar params
        if 'cider' in metric:
            cbar_params = {'cmap': 'Blues'} # 'vmin': MIN_CIDER, 'vmax': MAX_CIDER, 
        else:
            min_value, max_value = limits[abn]
            cbar_params = {'vmin': min_value, 'vmax': max_value, 'cmap': 'YlOrRd'}
            # cbar_params = {'vmin': 0, 'vmax': max_value + 0.2, 'cmap': 'YlOrRd'}
        
        plt.subplot(n_rows, n_cols, abn_i * n_cols + metric_col_j + 1)
        sns.heatmap(cube[metric_i], annot=True, square=True,
                    xticklabels=TICKS, yticklabels=TICKS, fmt='.3f', # robust=True,
                    cbar=True,
                    **cbar_params,
                   )

        title_metric = True # (abn_i == 0)
        include_ylabel = (metric_col_j == 0)
        include_xlabel = True # (abn_i == n_rows - 1)

        if title_metric:
            pretty_metric = get_pretty_metric(metric, metric_i=metric_i, include_range=True)
            plt.title(pretty_metric, fontsize=18)

        if include_xlabel:
            plt.xlabel('Generated', fontsize=18)

        if include_ylabel:
            plt.ylabel(f'{abn}\nGround Truth', fontsize=18)

# Attempting to optimize threshold

## Attempt 1

(failed)

In [ ]:
result = exp[0]
result

In [ ]:
# target1, target2 = (0, 0), (0, 1) # TN, FP (specificity)
target1, target2 = (1, 1), (1, 0) # TP, FN (precision)

arr1 = result.dists[target1]
arr2 = result.dists[target2]
arr1.shape, arr2.shape

In [ ]:
assert target1[0] == target1[1]
CORRECT = target1[0]
INCORRECT = 1 - CORRECT

merged = [(value, CORRECT) for value in arr1] + [(value, INCORRECT) for value in arr2]
merged = sorted(merged, reverse=bool(not CORRECT))
merged[:2], merged[-2:]

In [ ]:
all_threshs = []
denominator = 0 # TP + FN
numerator = 0 # TP
for value, label in merged:
    current_thresh = value
    if label == CORRECT: # add 1 TP
        numerator += 1

    denominator += 1

    all_threshs.append((current_thresh, numerator / denominator))
all_threshs[:5]

In [ ]:
max(all_threshs, key=lambda x: x[1])

In [ ]:
x, y = tuple(zip(*all_threshs))
plt.plot(x, y)

## Attempt 2

with sklearn
Failed

In [ ]:
from sklearn.metrics import precision_recall_curve as pr_curve

In [ ]:
pred, gt = tuple(zip(*merged))
pred = np.array(pred)
gt = np.array(gt)
pred.shape, gt.shape

In [ ]:
# pred /= 10 # CIDER re-scaling

In [ ]:
precision, recall, thresholds = pr_curve(gt, pred, pos_label=CORRECT)
precision.shape, recall.shape, thresholds.shape

In [ ]:
f1 = divide_arrays(2*precision*recall, precision + recall)
f1.shape

In [ ]:
best_idx = f1.argmax()
best_idx

In [ ]:
thresholds[best_idx], f1[best_idx], precision[best_idx], recall[best_idx]

## Attempt 3: accuracy/prec/recall

CheXpert 4-class classification task --> is a binary classification task in NLP scores
(i.e. NLP scores tell less information)

In [ ]:
exp = load_experiment_pickle('mimic-cardiomegaly')
exp

In [ ]:
result = exp.results[-1]
result.metric

In [ ]:
result.dists

In [ ]:
merged = [
    # Value, correct-or-not, original-key
    (value, 1, (0, 0)) for value in result.dists[(0, 0)]
] + [
    (value, 1, (1, 1)) for value in result.dists[(1, 1)]
] + [
    (value, 0, (0, 1)) for value in result.dists[(0, 1)]
] + [
    (value, 0, (1, 0)) for value in result.dists[(1, 0)]
]
merged = sorted(merged)
len(merged), merged[:3]

In [ ]:
n_correct = sum(1 for _, correct, _ in merged if correct)
n_incorrect = sum(1 for _, correct, _ in merged if not correct)
n_correct, n_incorrect

In [ ]:
def smart_division(a, b):
    if b == 0:
        return 0
    return a / b

In [ ]:
all_threshs = []

# At first, the threshold is at 0
# --> No negative predictions, all positive predictions
# --> TN = FN = 0
TP = sum(1 for _, correct, _ in merged if correct)
FP = sum(1 for _, correct, _ in merged if not correct)
TN, FN = 0, 0

total = len(merged)

assert TP + FP + FN + TN == total, f'Begin: {TP + FP + FN + TN} vs {total}'

for value, correct, _ in merged:
    current_thresh = value

    if correct:
        TP -= 1
        FN += 1
    else:
        TN += 1
        FP -= 1

    assert TP + FP + FN + TN == total, f'Thresh={value}: {TP + FP + FN + TN} vs {total}'
        
    acc = (TP + TN) / total
    prec = smart_division(TP, TP + FP)
    recall = smart_division(TP, TP + FN)
    f1 = smart_division(2*prec*recall, prec+recall)
    spec = smart_division(TN, TN + FP)
    npv = smart_division(TN, TN + FN)
    f1_neg = smart_division(2*npv*spec, spec+npv)
    CM = (TP, FN, FP, TN)

    all_threshs.append({
        'thresh': current_thresh,
        'acc': acc,
        'prec': prec,
        'recall': recall,
        'f1': f1,
        'npv': npv,
        'spec': spec,
        'f1_neg': f1_neg,
        'CM': CM,
    })
all_threshs[:1]

In [ ]:
max(all_threshs, key=lambda x: x['acc'])

In [ ]:
sl = lambda k: tuple(zip(*[(x['thresh'], x[k]) for x in all_threshs]))

In [ ]:
plt.figure(figsize=(6, 5))
keys = ('prec', 'recall', 'acc', 'f1') # 'f1', 
# keys = ('acc', )
# keys = ('npv', 'spec', 'f1_neg')
for k in keys:
    thresh, y = sl(k)
    plt.plot(thresh, y, label=k)
plt.legend()
plt.xlabel('Thresh')
plt.ylabel('Value')
plt.title('Optimize by')

In [ ]:
best = max(all_threshs, key=lambda x: x['acc'])
best

In [ ]:
def plot_cm(cm, title=None):
    TP, FN, FP, TN = cm
    ticks = ['Entailment', 'Contradiction']
    sns.heatmap([[TP, FN], [FP, TN]], annot=True, square=True, cmap='Blues',
                xticklabels=ticks, yticklabels=ticks, fmt=',',
               )
    plt.ylabel('Real')
    plt.xlabel('Scored by Metric')
    if title:
        plt.title(title)

In [ ]:
plot_cm(best['CM'], title=f'CM for {exp.abnormality} with {get_pretty_metric(result.metric)}')

## Attempt 4: use AUC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
def prepare_gt_pred_for_roc(result, metric_i=0, keys=None):
    pred = []
    gt = []
    if keys is None:
        keys = list(result.dists.keys())

    for a, b in keys:
        elements = result.dists[(a, b)]
        if elements.ndim == 2:
            elements = elements[metric_i] # BLEU case
        pred += list(elements)

        entailment = int(a == b)
        gt += [entailment] * len(elements)

    return gt, pred

In [ ]:
result = exp.results[-1]

In [ ]:
gt, pred = prepare_gt_pred_for_roc(result)

In [ ]:
fpr, tpr, thresholds = roc_curve(gt, pred)

J_stat = tpr - fpr
best_idx = J_stat.argmax()

thresholds[best_idx], J_stat[best_idx]

In [ ]:
roc = roc_auc_score(gt, pred)
roc

## Compute AUC for all abnormalities

In [ ]:
dataset_name = 'mimic'

In [ ]:
exp_by_abn_iu = load_experiments(dataset_name)
len(exp_by_abn_iu)

In [ ]:
show = True
target_sampler = None # 'random-gen_k500_n500'
target_groups = [0, 1] # [-2, -1, 0, 1]
# keys = [(0, 0), (0, 1), (1, 1), (1, 0)]
# keys = None
set_of_keys = [
    ((0, 0), (0, 1)),
    ((1, 1), (1, 0)),
]

final_records = []

for abnormality in CHEXPERT_DISEASES[1:]:
    if abnormality not in exp_by_abn_iu:
        continue
    exp = exp_by_abn_iu[abnormality]

    for result in tqdm(exp.results, desc=abnormality, disable=not show):
        if target_sampler is not None and result.sampler != target_sampler:
            continue
        if target_groups is not None and sorted(result.groups) != target_groups:
            continue
        
        for keys in set_of_keys:
            gt, pred = prepare_gt_pred_for_roc(result, keys=keys)
            roc = roc_auc_score(gt, pred)

            final_records.append((
                abnormality, result.metric, result.groups, result.sampler, keys, roc)
            )

            if result.metric == 'bleu':
                # HACK
                gt, pred = prepare_gt_pred_for_roc(result, metric_i=3, keys=keys)
                roc = roc_auc_score(gt, pred)
                final_records.append((
                    abnormality, f'{result.metric}-4', result.groups, result.sampler, keys,
                    roc,
                ))

len(final_records)

In [ ]:
final_records[:1]

In [ ]:
cols = ['disease', 'metric', 'groups', 'sampler', 'keys', 'roc']
df = pd.DataFrame(final_records, columns=cols)
df.head(2)

In [ ]:
Counter(df['groups']), Counter(df['sampler'])

In [ ]:
df = df.loc[df['groups'] == (0, 1)]
# df = df.loc[df['sampler'] == 'random-gen_k500_n500'] # IU
df = df.loc[df['sampler'] == 'random-gen_k50_n100'] # MIMIC
del df['sampler'], df['groups']
df.head(2)

In [ ]:
df_recall = df.loc[df['keys'] == ((1, 1), (1, 0))]
df_spec = df.loc[df['keys'] == ((0, 0), (0, 1))]
len(df_recall), len(df_spec)

In [ ]:
def get_renamer(replace_strs):
    def _rename_run(run_name):
        s = run_name
        for target, replace_with in replace_strs:
            s = re.sub(target, replace_with, s)
        return s
    return _rename_run

def bold(s):
    return '\textbf{' + s + '}'

shorten_cols = get_renamer([
    ('cider-IDF', 'C-D'),
    ('bleu-4', 'B-4'),
    (r'bleu\b', 'B-1'),
    ('rouge', 'R-L'),
    ('disease', 'Abnormality'),
])
def latexify_cols(col):
    return bold(shorten_cols(col))

In [ ]:
def df_to_table(df):
    cols = list(df.columns)
    metric_col = cols.index('metric')
    roc_col = cols.index('roc')
    df = df.groupby('disease').apply(lambda subdf: {
        row[metric_col]: row[roc_col]
        for row in list(subdf.values)
    }).apply(pd.Series)
    return df
def table_to_latex(table):
    s = table.reset_index().rename(columns=latexify_cols).to_latex(
        float_format='%.3f',
        escape=False,
        index=False,
        column_format='l' + 'c' * len(table.columns),
    )
    s = re.sub(r' +', ' ', s, flags=re.M)
    print(s)

In [ ]:
table_recall = df_to_table(df_recall)
table_recall

In [ ]:
table_spec = df_to_table(df_spec)
table_spec

In [ ]:
long_table = pd.concat([table_recall, table_spec], axis=1)
table_to_latex(long_table)

In [ ]:
def get_result(exp_by_abn, abnormality, metric,
               groups=[0, 1], sampler='random-gen_k500_n500'):
    if abnormality not in exp_by_abn:
        print(f'No exp for {abnormality}')
        return None, None
    groups = list(groups)
    exp = exp_by_abn[abnormality]

    for i, result in enumerate(exp.results):
        if sampler is not None and result.sampler != sampler:
            continue
        if groups is not None and sorted(result.groups) != groups:
            continue
        if result.metric != metric:
            continue
            
        return exp, i
    
    print('No experiment found with conditions')
    return exp, None

In [ ]:
exp, result_i = get_result(exp_by_abn_iu, 'Atelectasis', 'bleu')
exp

In [ ]:
plot_heatmap(exp, result_i=result_i, metric_i=3)

In [ ]:
plot_hists(exp, keys=[(0, 0), (0, 1)], result_i=result_i, metric_i=3, bins=50)

# Statistical tests

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu, f_oneway, kruskal

In [ ]:
# exp = load_experiment_pickle('mimic-cardiomegaly')
len(exp.results)

In [ ]:
plot_heatmap(exp, result_i=-1)

In [ ]:
EXP_I = -1
result = exp[EXP_I]
result.metric

In [ ]:
key1 = (0, 0)
key2 = (0, 1)
group1 = result.dists[key1]
group2 = result.dists[key2]
if result.metric == 'bleu':
    group1 = group1[0]
    group2 = group2[0]
group1.shape, group2.shape

In [ ]:
plot_hists(exp, [key1, key2], result_i=EXP_I, bins=50, range=(0, 1))

In [ ]:
r = mannwhitneyu(group1, group2)
r

In [ ]:
r = ttest_ind(group1, group2, equal_var=False)
r

In [ ]:
groups = [result.dists[k] for k in [(0, 0), (0, 1), (1, 0), (1, 1)]]

In [ ]:
anova = f_oneway(*groups)
anova

In [ ]:
kru = kruskal(*groups)
kru